In [30]:
import polars as pl
from pathlib import Path

In [31]:
DATA_PATH = Path("../dataset")

df = pl.read_csv(
    DATA_PATH / "germany_electricity_price_daily.csv",
    try_parse_dates=True
)

df = df.sort("Date")
df.head()

Date,Electricity_Price_EUR_per_MWh
date,f64
2015-01-01,22.34
2015-01-02,22.34
2015-01-03,22.34
2015-01-04,22.34
2015-01-05,36.18


## Baseline Models
### Naïve & Seasonal Naïve

In [32]:
df = df.rename({
    "Date": "date",
    "Electricity_Price_EUR_per_MWh": "y"
})

df = df.with_columns(
    pl.col("date").cast(pl.Date)
)

In [33]:
TEST_START = pl.date(2024, 1, 1)

train_df = df.filter(pl.col("date") < TEST_START)
test_df  = df.filter(pl.col("date") >= TEST_START)

len(train_df), len(test_df)

(3287, 725)

In [34]:
df = df.with_columns(
    pl.col("y").shift(1).alias("naive")
)

In [35]:
df = df.with_columns(
    pl.col("y").shift(7).alias("seasonal_naive_7")
)

In [36]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

eval_df = df.filter(pl.col("date") >= TEST_START).drop_nulls()

y_true = eval_df["y"].to_numpy()
y_naive = eval_df["naive"].to_numpy()
y_seasonal = eval_df["seasonal_naive_7"].to_numpy()

mae_naive = mean_absolute_error(y_true, y_naive)
mae_seasonal = mean_absolute_error(y_true, y_seasonal)

rmse_naive = np.sqrt(mean_squared_error(y_true, y_naive))
rmse_seasonal = np.sqrt(mean_squared_error(y_true, y_seasonal))

mape_naive = np.mean(np.abs((y_true - y_naive) / y_true)) * 100
mape_seasonal = np.mean(np.abs((y_true - y_seasonal) / y_true)) * 100

mae_naive, mae_seasonal, rmse_naive, rmse_seasonal, mape_naive, mape_seasonal

(20.872344827586208,
 27.09655172413793,
 np.float64(28.61233419349075),
 np.float64(38.9049402553951),
 np.float64(123.34692938456922),
 np.float64(193.8722594247381))

- Seasonal naive performs better than naive, showing clear weekly seasonality.
- These baselines will be used as reference for more complex models.

In [37]:
countries = {
    "austria": "austria_electricity_price_daily.csv",
    "germany": "germany_electricity_price_daily.csv",
    "switzerland": "switzerland_electricity_price_daily.csv",
    "france": "france_electricity_price_daily.csv",
    "poland": "poland_electricity_price_daily.csv",
    "estonia": "estonia_electricity_price_daily.csv",
}

In [38]:
def evaluate_baselines(df, test_start):
    df = df.with_columns([
        pl.col("y").shift(1).alias("naive"),
        pl.col("y").shift(7).alias("seasonal_naive_7"),
    ])
    
    eval_df = df.filter(pl.col("date") >= test_start).drop_nulls()
    
    y_true = eval_df["y"].to_numpy()
    y_naive = eval_df["naive"].to_numpy()
    y_seasonal = eval_df["seasonal_naive_7"].to_numpy()
    
    return {
        "mae_naive": mean_absolute_error(y_true, y_naive),
        "mae_seasonal": mean_absolute_error(y_true, y_seasonal),
        "rmse_naive": np.sqrt(mean_squared_error(y_true, y_naive)),
        "rmse_seasonal": np.sqrt(mean_squared_error(y_true, y_seasonal)),
        "mape_naive": np.mean(np.abs((y_true - y_naive) / y_true)) * 100,
        "mape_seasonal": np.mean(np.abs((y_true - y_seasonal) / y_true)) * 100,
    }

In [39]:
results = []

for country, file in countries.items():
    df = pl.read_csv(DATA_PATH / file, try_parse_dates=True)
    df = df.rename({"Date": "date", "Electricity_Price_EUR_per_MWh": "y"})
    df = df.sort("date")
    
    metrics = evaluate_baselines(df, TEST_START)
    metrics["country"] = country
    results.append(metrics)

In [40]:
baseline_results = pl.DataFrame(results)
baseline_results

mae_naive,mae_seasonal,rmse_naive,rmse_seasonal,mape_naive,mape_seasonal,country
f64,f64,f64,f64,f64,f64,str
16.1364,18.7068,22.61294,26.828859,25.454243,29.138782,"""austria"""
20.872345,27.096552,28.612334,38.90494,123.346929,193.872259,"""germany"""
13.008905,14.955368,18.391042,20.611866,35.823119,37.006315,"""switzerland"""
15.371462,25.347269,20.590942,34.040016,140.557778,84.570847,"""france"""
18.272028,22.906538,24.944024,30.345585,48.481325,78.520151,"""poland"""
31.931379,41.822469,55.1607,68.179462,71.786547,105.516075,"""estonia"""


In [41]:
TEST_START = pl.date(2024, 1, 1)

## Statistical Models
### Holt-Winters (ETS)

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pandas as pd

df_de = (
    pl.read_csv(DATA_PATH / "germany_electricity_price_daily.csv", try_parse_dates=True)
    .rename({"Date": "date", "Electricity_Price_EUR_per_MWh": "y"})
    .sort("date")
)

df_de_pd = df_de.to_pandas()
df_de_pd.set_index("date", inplace=True)

In [43]:
train_pd = df_de_pd[df_de_pd.index < "2024-01-01"]

hw_model = ExponentialSmoothing(
    train_pd["y"],
    trend="add",
    seasonal="add",
    seasonal_periods=7
).fit()

/Users/bl3ks/thenewtisvenv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [44]:
test_len = len(df_de_pd[df_de_pd.index >= "2024-01-01"])
hw_forecast = hw_model.forecast(test_len)

In [45]:
y_true = df_de_pd.loc["2024-01-01":, "y"].values
y_pred = hw_forecast.values

mae_hw = mean_absolute_error(y_true, y_pred)
rmse_hw = np.sqrt(mean_squared_error(y_true, y_pred))
mape_hw = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mae_hw, rmse_hw, mape_hw

(49.49095800880151,
 np.float64(57.99739781458339),
 np.float64(125.31325728423101))

- Holt-winters underperforms the weekly seasonal naive baseline for germany,
- Suggesting that simple seasonal persistence is hard to beat for daily prices.

### AutoARIMA (statsforecast)

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

df_de = (
    pl.read_csv(DATA_PATH / "germany_electricity_price_daily.csv", try_parse_dates=True)
    .rename({
        "Date": "ds",
        "Electricity_Price_EUR_per_MWh": "y"
    })
    .sort("ds")
)

df_de_pd = df_de.to_pandas()
df_de_pd["unique_id"] = "germany"

df_sf = df_de_pd[["unique_id", "ds", "y"]]

In [54]:
train_sf = df_sf[df_sf["ds"] < "2024-01-01"]
test_sf  = df_sf[df_sf["ds"] >= "2024-01-01"]

h = len(test_sf)

In [55]:
sf = StatsForecast(
    models=[AutoARIMA(season_length=7)],
    freq="D",
    n_jobs=1
)

sf.fit(train_sf)

StatsForecast(models=[AutoARIMA])

In [56]:
forecast = sf.predict(h=h)
y_pred = forecast["AutoARIMA"].values
y_true = test_sf["y"].values

In [57]:
mae_arima = mean_absolute_error(y_true, y_pred)
rmse_arima = np.sqrt(mean_squared_error(y_true, y_pred))
mape_arima = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mae_arima, rmse_arima, mape_arima

(30.235776626478795,
 np.float64(40.345270144091025),
 np.float64(145.4602662738201))

- AutoARIMA improves over naive but does not outperform the weekly seasonal naive baseline for Germany.


## Machine Learning Models
### Huber Regression 

In [ ]:
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

df_de = (
    pl.read_csv(DATA_PATH / "germany_electricity_price_daily.csv", try_parse_dates=True)
    .rename({"Date": "date", "Electricity_Price_EUR_per_MWh": "y"})
    .sort("date")
)

df_de = df_de.with_columns([
    pl.col("y").shift(1).alias("lag_1"),
    pl.col("y").shift(7).alias("lag_7"),
    pl.col("y").shift(14).alias("lag_14"),
    pl.col("y").rolling_mean(7).alias("roll_mean_7"),
    pl.col("y").rolling_std(7).alias("roll_std_7"),
])

df_de = df_de.drop_nulls()

In [59]:
train_df = df_de.filter(pl.col("date") < TEST_START)
test_df  = df_de.filter(pl.col("date") >= TEST_START)

features = ["lag_1", "lag_7", "lag_14", "roll_mean_7", "roll_std_7"]

X_train = train_df.select(features).to_numpy()
y_train = train_df["y"].to_numpy()

X_test = test_df.select(features).to_numpy()
y_test = test_df["y"].to_numpy()

In [60]:
huber = HuberRegressor()
huber.fit(X_train, y_train)

y_pred = huber.predict(X_test)

In [61]:
mae_huber = mean_absolute_error(y_test, y_pred)
rmse_huber = np.sqrt(mean_squared_error(y_test, y_pred))
mape_huber = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

mae_huber, rmse_huber, mape_huber

(17.935196530263532,
 np.float64(24.53770947030222),
 np.float64(144.32123365472273))

- Huber-Regression outperforms the weekly seasonal naive baseline for Germany.


In [62]:
def evaluate_huber(df, test_start):
    df = df.with_columns([
        pl.col("y").shift(1).alias("lag_1"),
        pl.col("y").shift(7).alias("lag_7"),
        pl.col("y").shift(14).alias("lag_14"),
        pl.col("y").rolling_mean(7).alias("roll_mean_7"),
        pl.col("y").rolling_std(7).alias("roll_std_7"),
    ]).drop_nulls()
    
    train_df = df.filter(pl.col("date") < test_start)
    test_df  = df.filter(pl.col("date") >= test_start)
    
    features = ["lag_1", "lag_7", "lag_14", "roll_mean_7", "roll_std_7"]
    
    X_train = train_df.select(features).to_numpy()
    y_train = train_df["y"].to_numpy()
    
    X_test = test_df.select(features).to_numpy()
    y_test = test_df["y"].to_numpy()
    
    model = HuberRegressor()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    return {
        "mae_huber": mean_absolute_error(y_test, y_pred),
        "rmse_huber": np.sqrt(mean_squared_error(y_test, y_pred)),
        "mape_huber": np.mean(np.abs((y_test - y_pred) / y_test)) * 100,
    }

In [63]:
huber_results = []

for country, file in countries.items():
    df = (
        pl.read_csv(DATA_PATH / file, try_parse_dates=True)
        .rename({"Date": "date", "Electricity_Price_EUR_per_MWh": "y"})
        .sort("date")
    )
    
    metrics = evaluate_huber(df, TEST_START)
    metrics["country"] = country
    huber_results.append(metrics)

In [64]:
huber_results_df = pl.DataFrame(huber_results)
huber_results_df

mae_huber,rmse_huber,mape_huber,country
f64,f64,f64,str
13.59461,18.753448,22.841303,"""austria"""
17.935197,24.537709,144.321234,"""germany"""
11.268638,15.448784,33.303047,"""switzerland"""
14.576476,18.833351,124.258876,"""france"""
15.295268,20.340829,54.009128,"""poland"""
27.148975,45.713036,71.684166,"""estonia"""


- Huber-Regression improves over baselines for most countries, showing the value of lag-based features.